In [1]:
from sklearn.datasets import load_diabetes

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
X,y = load_diabetes(return_X_y=True)

In [3]:
print(X.shape)
print(y.shape)

(442, 10)
(442,)


In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [5]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [6]:
print(reg.coef_)
print(reg.intercept_)

[  -9.15865318 -205.45432163  516.69374454  340.61999905 -895.5520019
  561.22067904  153.89310954  126.73139688  861.12700152   52.42112238]
151.88331005254167


In [7]:
y_pred = reg.predict(X_test)
r2_score(y_test,y_pred)

0.4399338661568968

In [8]:
X_train.shape

(353, 10)

In [9]:
# Batch Gradient Descent
class GDRegressor:
    
    def __init__(self,learning_rate=0.01,epochs=100):
        
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        
    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            # update all the coef and the intercept
            y_hat = np.dot(X_train,self.coef_) + self.intercept_
            #print("Shape of y_hat",y_hat.shape)
            intercept_der = -2 * np.mean(y_train - y_hat)
            self.intercept_ = self.intercept_ - (self.lr * intercept_der)
            
            coef_der = -2 * np.dot((y_train - y_hat),X_train)/X_train.shape[0]
            self.coef_ = self.coef_ - (self.lr * coef_der)
        
        print(self.intercept_,self.coef_)
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [10]:
gdr = GDRegressor(epochs=1000,learning_rate=0.5)

In [11]:
gdr.fit(X_train,y_train)

152.01351687661833 [  14.38990585 -173.7235727   491.54898524  323.91524824  -39.32648042
 -116.01061213 -194.04077415  103.38135565  451.63448787   97.57218278]


In [12]:
y_pred = gdr.predict(X_test)

In [13]:
r2_score(y_test,y_pred)


0.4534503034722803

In [14]:
# Stochastic Gradient Descent
class SGDRegressor:
    
    def __init__(self,learning_rate=0.01,epochs=100):
        
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        
    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            for j in range(X_train.shape[0]):
                idx = np.random.randint(0,X_train.shape[0])
                
                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                
                intercept_der = -2 * (y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)
                
                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)
        
        print(self.intercept_,self.coef_)
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [15]:
sgd = SGDRegressor(learning_rate=0.01,epochs=40)

In [17]:
import time
start = time.time()
sgd.fit(X_train,y_train)
print("The time taken is",time.time() - start)

155.59262478363033 [  60.2544689   -36.36314215  318.06225897  223.05008872   30.51221612
  -11.38393564 -155.48585965  130.16281826  290.6737637   131.22437983]
The time taken is 0.2539670467376709


In [18]:
y_pred = sgd.predict(X_test)

In [19]:
r2_score(y_test,y_pred)

0.41722783925372

In [20]:
from sklearn.linear_model import SGDRegressor

In [23]:
reg = SGDRegressor(max_iter=200,learning_rate='constant',eta0=0.01)

In [24]:
reg.fit(X_train,y_train)

SGDRegressor(learning_rate='constant', max_iter=200)

In [25]:
y_pred = reg.predict(X_test)

In [26]:
r2_score(y_test,y_pred)

0.4419799651948776

In [42]:
import random

class MBGDRegressor:
    
    def __init__(self,batch_size,learning_rate=0.01,epochs=100):
        
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        
    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            
            for j in range(int(X_train.shape[0]/self.batch_size)):
                
                idx = random.sample(range(X_train.shape[0]),self.batch_size)
                
                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                #print("Shape of y_hat",y_hat.shape)
                intercept_der = -2 * np.mean(y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)

                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)
        
        print(self.intercept_,self.coef_)
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [107]:
mbr = MBGDRegressor(batch_size=int(X_train.shape[0]/50),learning_rate=0.01,epochs=100)

In [108]:
mbr.fit(X_train,y_train)

148.70016581380176 [  26.33527377 -142.27983914  453.30173711  306.82154016  -20.77004301
  -84.65802955 -197.87656804  119.68949049  404.68880048  123.96909505]


In [109]:
y_pred = mbr.predict(X_test)

In [110]:
r2_score(y_test,y_pred)

0.45165545593227074

In [111]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor(learning_rate='constant',eta0=0.1)

In [112]:
batch_size = 35

for i in range(100):
    
    idx = random.sample(range(X_train.shape[0]),batch_size)
    sgd.partial_fit(X_train[idx],y_train[idx])

In [113]:
sgd.coef_

array([  59.63776274,  -72.36317566,  369.02300972,  248.0157716 ,
         20.37740062,  -27.92544729, -178.22829398,  130.40002623,
        326.74269546,  121.07630947])

In [114]:
sgd.intercept_

array([164.43063204])

In [115]:
y_pred = sgd.predict(X_test)

In [116]:
r2_score(y_test,y_pred)

0.40850043711781947